# Setup

In [ ]:
import pandas as pd
from case_study_utils import CorrCommunity
from case_study_ui import show_communities, show_df
import psycopg2
from sqlalchemy import create_engine
from corr_analysis import get_aligned_results
from time_point import T_GRANU
from coordinate import S_GRANU

pd.set_option("display.max_rows", None)

# create the connection to database
db_path = "postgresql://yuegong@localhost/chicago_1m_zipcode"
data_path = "/home/cc/resolution_aware_spatial_temporal_alignment/data/chicago_open_data_1m/"
db = create_engine(db_path)
conn_copg2 = psycopg2.connect(db_path)
cur = conn_copg2.cursor()

# Load and Show Correlations

The input is the asthma dataset. We align the input with tables from Chicago open data and compute correlations. Tables from chicago open data originally have the spatial granularity of geo-coordinate. We aggregate them to the zipcode level and apply aggregate functions "avg" and "count". For example, if you see an attribute named "avg_basketball_courts", it means the original attribute is "basketball_courts" and function "avg" is applied. The attribute after aggregation is named "avg_basketball_courts". 

`missing_ratio_o2` is the fraction of missing values in the original attribute before any aggregation.

`r_val` is the correlation coefficient (we use pearson's correlation coefficient).

`r_abs` is the absoluate value of correlation coefficient.

`samples` is the number of zipcodes matched between two datasets.

In the displayed dataframe, you can perform sorting on one dimension, filtering rows using keywords, and etc.

In [5]:
corrs = pd.read_csv('./corr_asthma.csv')
show_df(corrs)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

# Check original data and more metadata information

To check the original data and get more context of the data, you can either search the table name in Chicago open data portal (https://data.cityofchicago.org/) or use the following api(copy the table id as the input).

In [6]:
dataset_id = 'me59-5fac'
df = pd.read_csv(f'{data_path}/{dataset_id}.csv')
display(df.head())

,creation_date,status,completion_date,service_request_number,type_of_service_request,what_is_the_nature_of_this_code_violation_,street_address,zip_code,x_coordinate,y_coordinate,ward,police_district,community_area,latitude,longitude,location
0,2018-12-18T00:00:00.000,Open,NaN,18-03388428,Sanitation Code Violation,Other,3200 W ADDISON ST,60618.0,1.154414e+06,1.923763e+06,33.0,17.0,16.0,41.946631,-87.707822,"(41.946630935842, -87.707821919321)"
1,2018-12-18T00:00:00.000,Open - Dup,NaN,18-03389154,Sanitation Code Violation,Dumpster not being emptied,11109 S EMERALD AVE,60628.0,1.173296e+06,1.831190e+06,34.0,22.0,49.0,41.692203,-87.641161,"(41.692202556477, -87.641160530998)"
2,2018-12-18T00:00:00.000,Open,NaN,18-03388637,Sanitation Code Violation,Other,1419 N LARAMIE AVE,60651.0,1.141443e+06,1.908946e+06,37.0,25.0,25.0,41.906222,-87.755868,"(41.906222326111, -87.75586824686)"
3,2018-12-18T00:00:00.000,Open,NaN,18-03387752,Sanitation Code Violation,Other,5706 S NARRAGANSETT AVE,60638.0,1.134696e+06,1.866027e+06,13.0,8.0,56.0,41.788565,-87.781667,"(41.788565462039, -87.781666801009)"
4,2018-12-18T00:00:00.000,Completed,2018-12-18T00:00:00.000,18-03387403,Sanitation Code Violation,Other,6714 S MERRILL AVE,60649.0,1.191700e+06,1.860873e+06,5.0,3.0,43.0,41.773231,-87.572821,"(41.773230851405, -87.572820721701)"


# Check the aligned result

if you are interested in one correlation and want to check the aligned result used to calculate the correlation, you can use the following api. The input to this api is the index of the correlation (the first column in the correlation result).

In [7]:
corr_idx = 10
row = corrs.loc[corr_idx]
df = get_aligned_results(cur, row, None, S_GRANU.ZIPCODE)
display(df)

,val,encAsthmaExac,avg_basketball_backboards
0,60634,4.0,0.500000
1,60612,17.0,1.285714
2,60611,4.0,0.000000
3,60605,7.0,0.000000
4,60622,2.0,1.384615
5,60644,4.0,1.833333
6,60640,2.0,0.307692
7,60613,4.0,0.000000
8,60618,11.0,1.176471
9,60649,693.0,2.888889
